In [3]:
import os
import sys
import cv2
import csv
import tqdm
import pandas as pd
import random
import numpy as np
import time
from skimage.morphology import skeletonize, medial_axis
from matplotlib import pyplot as plt

In [4]:

def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    # return the intersection over union value
    return iou

In [5]:
CSV_SORT_PATH = "C:/Users/benja\Desktop/NN_alive_dead_training_vids/Sorted/345_sort_forward.csv"
df = pd.read_csv(CSV_SORT_PATH,names=('frame', 'x1', 'y1', 'x2', 'y2','label','delta'))
df['catagory'] = 'alive'
#df['X']=df[['x1','x2']].mean(axis=1)
#df['Y']=df[['y1','y2']].mean(axis=1)
#df = df[['frame', 'X', 'Y','label']]
uniqueFrames = df["frame"].unique()
uniqueIDs = df["label"].unique()

In [76]:
vc = df.label.value_counts()
test = vc[vc > 25].index.tolist()

In [29]:
test = vc[vc > 50].index.tolist()

In [79]:
print(df)

        frame           x1          y1           x2          y2   label  \
0         2.0   982.000000  382.000000  1005.000000  403.000000     4.0   
1         2.0  1049.000000  425.000000  1070.000000  441.000000     3.0   
2         2.0   551.000000  648.000000   580.000000  673.000000     2.0   
3         2.0  1226.000000  936.000000  1249.000000  956.000000     1.0   
4         3.0   980.218639  380.813278  1006.781361  404.186722     4.0   
...       ...          ...         ...          ...         ...     ...   
12034  1729.0   741.573971  540.456008   759.288132  569.587296  9233.0   
12035  1729.0   825.220313  213.328701   845.957308  233.592129  9213.0   
12036  1729.0  1056.100510  511.042497  1086.822301  542.581513  9154.0   
12037  1729.0  1199.484941  673.997831  1220.798998  699.398399  8780.0   
12038  1729.0   812.103997  249.358885   841.149299  279.953252  8025.0   

       delta catagory  
0        NaN    alive  
1        NaN    alive  
2        NaN    alive  
3  

In [26]:
result = [a for a in df.label if a in test]

In [27]:
print(result)

[6093.0, 6093.0, 6093.0, 6093.0, 6093.0, 6093.0, 6093.0, 6093.0, 6093.0, 6093.0, 6093.0, 6279.0, 6093.0, 6279.0, 6093.0, 6279.0, 6093.0, 6279.0, 6093.0, 6279.0, 6093.0, 6279.0, 6093.0, 6279.0, 6093.0, 6279.0, 6093.0, 6279.0, 6093.0, 6279.0, 6093.0, 6279.0, 6093.0, 6279.0, 6093.0, 6279.0, 6093.0, 6279.0, 6093.0, 6279.0, 6093.0, 6093.0, 6093.0, 6093.0, 6279.0, 6093.0, 6279.0, 6093.0, 6279.0, 6093.0, 6279.0, 6093.0, 6279.0, 6093.0, 6279.0, 6093.0, 6279.0, 6093.0, 6279.0, 6093.0, 6279.0, 6093.0, 6279.0, 6093.0, 6279.0, 6093.0, 6279.0, 6093.0, 6279.0, 6093.0, 6279.0, 6279.0, 6279.0, 6279.0, 6279.0, 6279.0, 6279.0, 6279.0, 6093.0, 6279.0, 6093.0, 6279.0, 6093.0, 6279.0, 6093.0, 6093.0, 6734.0, 6093.0, 6734.0, 6093.0, 6734.0, 6279.0, 6093.0, 6279.0, 6093.0, 6279.0, 6093.0, 6279.0, 6093.0, 6734.0, 6279.0, 6093.0, 6734.0, 6279.0, 6093.0, 6734.0, 6279.0, 6093.0, 6279.0, 6093.0, 6279.0, 6093.0, 6279.0, 6093.0, 6734.0, 6279.0, 6093.0, 6734.0, 6279.0, 6093.0, 6734.0, 6279.0, 6093.0, 6734.0, 6279.0,

In [31]:
vc = df.label.counts()

AttributeError: 'Series' object has no attribute 'counts'

In [53]:
csv_outputs = []
for ID in test:        
    filtval = df['label'] ==ID
    interim = df[filtval]
    
    
    csv_outputs.append(interim)
#print(csv_outputs)    

[        frame           x1          y1           x2          y2   label  delta
1038    655.0  1127.333036  859.658158  1157.316667  895.611481  7684.0    NaN
1043    656.0  1126.102097  862.802014  1156.435228  897.086030  7684.0    NaN
1049    657.0  1126.630532  861.707296  1156.912061  896.967331  7684.0    NaN
1054    658.0  1126.788080  864.055564  1158.093955  902.943786  7684.0    NaN
1060    659.0  1128.479109  866.302757  1165.900105  911.771005  7684.0    NaN
...       ...          ...         ...          ...         ...     ...    ...
11988  1722.0  1115.522024  863.606231  1134.502922  900.176336  7684.0    NaN
11995  1723.0  1116.013253  863.332443  1135.240714  899.967375  7684.0    NaN
12004  1724.0  1115.968768  863.875057  1135.102674  900.545522  7684.0    NaN
12012  1725.0  1116.242897  863.686054  1134.760995  899.851641  7684.0    NaN
12021  1726.0  1117.100941  861.832442  1135.185231  896.796368  7684.0    NaN

[1033 rows x 7 columns],         frame          x1

In [35]:
filtval = df['label'] == 7684.0
interim = df[filtval]
interimD = []
interim2 = np.asarray(interim)
fill = 0
deadcount = 0
for row in interim2:
    frameNA, x1A, y1A, x2A, y2A, labelA, deltaA, catagoryA, *_ = row
    if fill > 1:
        boxA = [x1A, y1A, x2A, y2A]
        boxB = [x1B, y1B, x2B, y2B]
        deltaA = bb_intersection_over_union(boxA, boxB)  
        if deltaA > 0.95:
            deadcount += 1
    if deadcount > 5:
        catagoryA = 'dead'
        #print(frameNA)
    newRow = [frameNA, x1A, y1A, x2A, y2A, labelA, deltaA, catagoryA]
    interimD.append(newRow)
    frameNB, x1B, y1B, x2B, y2B,labelB, deltaB, catagoryB, *_ = row
    fill +=1
    

0.8210288061907879


In [36]:
vc = df.label.value_counts()
test = vc[vc > 25].index.tolist()
csv_outputs = []
#csv_outputs = pd.DataFrame(csv_outputs, columns = ['frame', 'x1', 'y1', 'x2', 'y2','label','delta','catagory'])

for ID in test:        
    filtval = df['label'] ==ID
    interim = df[filtval]
    interimD = []
    interim2 = np.asarray(interim)
    fill = 0
    deadcount = 0
    for row in interim2:
        frameNA, x1A, y1A, x2A, y2A, labelA, deltaA, catagoryA, *_ = row
        if fill > 1:
            boxA = [x1A, y1A, x2A, y2A]
            boxB = [x1B, y1B, x2B, y2B]
            deltaA = bb_intersection_over_union(boxA, boxB)  
            if deltaA > 0.95:
                deadcount += 1
        if deadcount > 5:
            catagoryA = 'dead'
            #print(frameNA)
        newRow = frameNA, x1A, y1A, x2A, y2A, labelA, deltaA, catagoryA
        interimD.append(newRow)
        frameNB, x1B, y1B, x2B, y2B,labelB, deltaB, catagoryB, *_ = row
        fill +=1
    interimD = pd.DataFrame(interimD, columns = ['frame', 'x1', 'y1', 'x2', 'y2','label','delta','catagory'])
    csv_outputs.append(interimD)
print('done')    

done


In [105]:
check = pd.DataFrame(csv_outputs)

C:\Users\benja\Anaconda3\lib\site-packages\pandas\core\internals\construction.py:309: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  values = np.array([convert(v) for v in values])


In [118]:
print(csv_outputs)

[[(655.0, 1127.3330361184178, 859.6581582136497, 1157.3166671827692, 895.6114809849647, 7684.0, nan, 'alive'), (656.0, 1126.1020968781406, 862.8020139149119, 1156.4352284466934, 897.0860296595283, 7684.0, nan, 'alive'), (657.0, 1126.6305321394298, 861.7072958122375, 1156.9120614685376, 896.9673312133381, 7684.0, 0.9366107059593116, 'alive'), (658.0, 1126.7880800027651, 864.0555639152583, 1158.093954804765, 902.9437862513724, 7684.0, 0.7718667873785747, 'alive'), (659.0, 1128.4791092724424, 866.3027568489663, 1165.900104988208, 911.7710050851124, 7684.0, 0.5996933353493316, 'alive'), (660.0, 1118.7231259398918, 868.3192743686744, 1159.1965596633167, 917.0753511135434, 7684.0, 0.5780591332180254, 'alive'), (661.0, 1112.6294028714012, 866.7772387277715, 1156.608243236072, 913.8403552877918, 7684.0, 0.7486248166495626, 'alive'), (662.0, 1113.4805197805558, 870.26216074107, 1156.6619084240965, 919.7024910478948, 7684.0, 0.8117876426136404, 'alive'), (663.0, 1109.043730958457, 871.9394565639

In [121]:
large_df = pd.concat(csv_outputs, ignore_index=True)    


In [122]:
print(large_df)

        frame           x1          y1           x2          y2   label  \
0       655.0  1127.333036  859.658158  1157.316667  895.611481  7684.0   
1       656.0  1126.102097  862.802014  1156.435228  897.086030  7684.0   
2       657.0  1126.630532  861.707296  1156.912061  896.967331  7684.0   
3       658.0  1126.788080  864.055564  1158.093955  902.943786  7684.0   
4       659.0  1128.479109  866.302757  1165.900105  911.771005  7684.0   
...       ...          ...         ...          ...         ...     ...   
10279  1007.0   947.014569  241.031045   984.051476  252.445169  8928.0   
10280  1008.0   946.146118  241.198027   982.548553  252.650056  8928.0   
10281  1009.0   946.409127  241.254914   982.075922  252.731080  8928.0   
10282  1010.0   946.806084  241.232322   984.250520  252.802307  8928.0   
10283  1011.0   947.191127  242.578955   982.259102  254.737071  8928.0   

          delta catagory  
0           NaN    alive  
1           NaN    alive  
2      0.936611   

In [7]:
threshold = 45
vc = df.label.value_counts()
test = vc[vc > threshold].index.tolist()
csv_outputs = []
deadboxes = []
for ID in test:        
    filtval = df['label'] ==ID
    interim = df[filtval]
    interimD = []
    interim2 = np.asarray(interim)
    fill = 0
    deadcount = 0
    for row in reversed(interim2):
        frameNA, x1A, y1A, x2A, y2A, labelA, deltaA, catagoryA, *_ = row
        if fill > 1:
            boxA = [x1A, y1A, x2A, y2A]
            boxB = [x1B, y1B, x2B, y2B]
            deltaA = bb_intersection_over_union(boxA, boxB)  
            if deltaA < 0.95:
                deadcount += 1
                #print(deadcount)
            #print(fill)
            if deadcount > 10:
                catagoryA = 'dead'
            if deadcount == 11:
                csv_outputs.append(frameNA)
                
                #print(deathtime)
                #print(frameNA)
            #newRow = [frameNA, x1A, y1A, x2A, y2A, labelA, deltaA, catagoryA]
            #interimD.append(newRow)
        frameNB, x1B, y1B, x2B, y2B,labelB, deltaB, catagoryB, *_ = row
        fill +=1
        #interimD.append(deathtime)
    #interimD = pd.DataFrame(interimD, columns = ['deathframe'])
    #csv_outputs.append(interimD)
#large_df = pd.concat(csv_outputs, ignore_index=True)  
csv_outputs = pd.DataFrame(csv_outputs, columns = ['deathframe'])
OUT_PATH = 'C:/Users/benja/Desktop/NN_alive_dead_training_vids/Sorted_Deathtimes/345_sorted_AD.csv'
pd.DataFrame(csv_outputs).to_csv(OUT_PATH, mode='a', header=False, index=None)

In [8]:
print(csv_outputs)

    deathframe
0       1028.0
1       1073.0
2       1072.0
3       1042.0
4       1041.0
5       1040.0
6       1029.0
7       1074.0
8       1073.0
9       1072.0
10      1079.0
11      1078.0
12      1039.0
13      1038.0
14      1037.0
15      1036.0
16      1035.0
17      1034.0
18      1039.0
19      1040.0
20      1039.0
21      1038.0
22      1037.0
23      1036.0
24      1035.0
25       939.0
26       938.0
27       937.0
28       647.0
29       646.0
30       547.0
31       630.0
32       509.0
33       622.0
34       788.0
35       732.0
36       724.0
37       603.0
38       587.0
39       507.0


In [9]:
OUT_PATH = 'C:/Users/benja/Desktop/NN_alive_dead_training_vids/Sorted_Deathtimes/345_sorted_AD.csv'
pd.DataFrame(csv_outputs).to_csv(OUT_PATH, mode='a', header=False, index=None)


In [ ]:
deadboxes = []
if x is not in deadboxes:
    